# RFdc

The RF Data Converter is the mechanism to convert digital data to analog data while performing operations in the digital domain.
Besides high-speed ADCs and high-speed DACs, there are many additional capabilities, such as digital up-or down conversion, decimation and interpolation and digital filtering.
This Notebook provides example usages for some common use-cases.

In [0]:
import qiclib as ql

platform = "slot1-platform"  # The Name or IP-address of your platform
qic = ql.QiController(platform)

[QiController] qiclib version: undetectable
[QiController] Establishing remote connection to slot1-platform...
[QiController] Detected QiController running on ZCU216 board
[QiController] Firmware build time: 20.10.2023 15:21:58 (Revision 825FCA19)
[QiController] Firmware with 6 digital unit cells detected.


## Checking the status

The DACs and ADCs are organized in tiles and blocks.
In the ZCU216, there are four tiles in total and four blocks per tile.

On the fontend board, the ADC tiles are labeled 224, 225, 226, 227 and correspond to tiles 0 to 3 respectively for usage in the `rfdc` component.
Equivalently, the DAC tiles are labeled 228, 229, 230, 231 and correspond to tiles 0 to 3 for usage in the `rfdc` component.

On the board, the label is given in the form `x_yyy`. `x` corresponds to the block, `yyy` to the tile-number.

The following cell prints status information about each ADC and DAC.

In [0]:
print("ADC status")
for tile in range(4):
    print(f"  Tile {tile}:")
    for block in range(4):
        status = qic.rfdc.adc(0, 0).status
        print(
            f"    block {block}: fs={status.sampling_frequency} GHz, clocks enabled = {status.data_path_clocks_enabled}"
        )

print()
print("DAC status")
for tile in range(4):
    print(f"  Tile {tile}")
    for block in range(4):
        status = qic.rfdc.dac(0, 0).status
        print(
            f"    block {block}: fs={status.sampling_frequency} GHz, clocks enabled = {status.data_path_clocks_enabled}"
        )

ADC status
  Tile 0:
    block 0: fs=2 GHz, clocks enabled = True
    block 1: fs=2 GHz, clocks enabled = True
    block 2: fs=2 GHz, clocks enabled = True
    block 3: fs=2 GHz, clocks enabled = True
  Tile 1:
    block 0: fs=2 GHz, clocks enabled = True
    block 1: fs=2 GHz, clocks enabled = True
    block 2: fs=2 GHz, clocks enabled = True
    block 3: fs=2 GHz, clocks enabled = True
  Tile 2:
    block 0: fs=2 GHz, clocks enabled = True
    block 1: fs=2 GHz, clocks enabled = True
    block 2: fs=2 GHz, clocks enabled = True
    block 3: fs=2 GHz, clocks enabled = True
  Tile 3:
    block 0: fs=2 GHz, clocks enabled = True
    block 1: fs=2 GHz, clocks enabled = True
    block 2: fs=2 GHz, clocks enabled = True
    block 3: fs=2 GHz, clocks enabled = True

DAC status
  Tile 0
    block 0: fs=4 GHz, clocks enabled = True
    block 1: fs=4 GHz, clocks enabled = True
    block 2: fs=4 GHz, clocks enabled = True
    block 3: fs=4 GHz, clocks enabled = True
  Tile 1
    block 0: fs=4 G

## Getting and adapting the mixer frequency

The RFdc comes with a mixer that is steered with a 48-bit NCO. The frequency of this NCO can be adjusted in the range of `-fs/2` to `+fs/2`.

For this, the DAC at tile/block index 0/2 (i.e. 2_228 on the board) is observed. Initially, its frequency is zero

In [0]:
qic.rfdc.dac(0, 2).mixer_frequency

0.0

The frequency can be changed using the `set_mixer_frequency` method. It expects the type of the converter (either "dac" or "adc") and the tile / block configuration as well as the frequency in Hz

In [0]:
qic.rfdc.dac(0, 2).mixer_frequency = 500e6

Finally, we obtain the changed frequency. Due to numerical inaccuracies, this frequency might different than the frequency that was set.

In [0]:
qic.rfdc.dac(0, 2).mixer_frequency

500000000.0

Using a simple `QiJob`, we can verify that the frequency was set.
The DAC at tile 0, block 2 is connected to the readout output of the Digital Unit Cell with index 0.

The purpose of the `QiJob` is to play a continuous wave. The length that is passed to the `QiPulse` is normally a floating-point value describing the duration in seconds. The special string "cw", however, enables playing out a continuous wave.
Running the same program with `length="off"` disables the wave.

The frequency that is passed to the `QiPulse` is that of a secondary, internal NCO. This NCO is used for fast frequency sweeps, however the bandwidth is limited to 1 GHz (i.e. -500 MHz to 500 MHz).

The output can be connected to a spectrum analyzer or an oscilloscope where a continuous sine-wave with a frequency of 500 MHz can be observed.

In [0]:
from qiclib.code import *

with QiJob() as job:
    q = QiCells(1)
    PlayReadout(q[0], QiPulse(length="cw", frequency=0))

In [0]:
job.run(qic)

HTML(value="<table style='width:100%'><tr><td> (0/1) </td><td>&#9992; -?-    </td><td>&#128336;  --:--:--   (e…

IntProgress(value=0, description='Averages', layout=Layout(width='95%'), max=1)